In [ ]:
import warnings
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
#import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

warnings.filterwarnings('ignore')

In [ ]:
# CONSTANTES
# Root directory of the project
BASE_DIR = "C:\\Users\\hp\\Desktop\\Data Science\\Projet Alain\\Curve_Reconizer"
DEFAULT_LOGS_DIR = os.path.join(BASE_DIR, "logs")
MODEL_DIR = os.path.join(BASE_DIR, "logs")
WEIGHTS_PATH = "C:\\Users\\hp\\Desktop\\Data Science\\Projet Alain\\Curve_Reconizer\\logs\\object20210629T2054\\mask_rcnn_object_0020.h5"   # change it


In [ ]:
class CustomConfig(Config):
    '''Configuration pour entrainer notre dataset customisé'''

    NAME = "object"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 2  # Nombre de classes = Background + success et failure
    STEPS_PER_EPOCH = 10 # Nombre d'étapes d'apprentissage par epoch
    DETECTION_MIN_CONFIDENCE = 0.9 # Sauter les détections avec un niveau de confiance < 90

In [ ]:
class CustomDataset(utils.Dataset):
    
    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "success")
        self.add_class("object", 2, "failure")

        # Train Dataset or Validation Dataset
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations1 = json.load(open('C:\\Users\\hp\\Desktop\\Data Science\\Projet Alain\\Curve_Reconizer\\datasets\\train\\train_annoted.json'))
        annotations = list(annotations1.values())
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # Obtention des coordonnées x, y des points des polygones qui composent le contour de chaque instance d'objet.
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"success": 1,"failure": 2}

            num_ids = [name_dict[a] for a in objects]
            print("numids",num_ids)

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )
            
    def load_mask(self, image_id):
        """Générer des masques d'instance pour une image.
       Returns:
        masks:  Un tableau bool de forme [hauteur, largeur, nombre d'instances] avec un masque par instance.
        class_ids: un tableau 1D d'ID de classe des masques d'instance.
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convertir les polygones en un masque bitmap de forme
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])], dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)


    def image_reference(self, image_id):
        """Retourne le chemin de l'image"""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
# Inference Model
TEST_MODE = "inference"
BASE_DIR = "C:\\Users\\hp\\Desktop\\Data Science\\Projet Alain\\Curve_Reconizer\\datasets"

def get_ax(rows=1, cols=1, size=16):
    """
      Retourne un tableau d'axes Matplotlib à utiliser dans toutes les visualisations du carnet.  
      Fournit un point central pour contrôler la taille des graphiques. 
      Ajustez l'attribut size pour contrôler la taille des images à rendre.
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax



# Chargement du validation set
dataset = CustomDataset()
dataset.load_custom(BASE_DIR, "val")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Création du modèle en mode inférence
config = CustomConfig()
model = modellib.MaskRCNN(mode='inference', model_dir=MODEL_DIR, config=config)

In [ ]:
# Chargement des poids 
print('Chargement des poids...')
weights_path = WEIGHTS_PATH
model.load(weightd_path, by_name=True)

### Test avec une image non présente dans la BD

In [ ]:
new_image = 'Chemin de la nouvelle image'
image1 = mpimg.imread(new_image)
results = model.detect([image1], verbose=1)

# Affichage du résultat
ax = get_ax(1)
r1 = results[0]
visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
dataset.class_names, r1['scores'], ax=ax, title="Predictions")